In [13]:
#Add all the imports here
import yfinance as yf
from datetime import datetime, timedelta, time
import pandas as pd
import numpy as np
from itertools import combinations
import itertools
import plotly.graph_objects as go
import time


In [14]:
#To get data from finanace
def get_data(symbol, date,interval):
  ticker = yf.Ticker(symbol)
  start_date = datetime.strptime(date, "%Y-%m-%d")
  end_date = start_date + timedelta(days=1)
  try:
    data = ticker.history(start=start_date, end=end_date, interval=interval);
  except Exception as e:
    return None
  return data

In [15]:
#Graph helpers

def create_candlestick_figure(ticker_symbol,date):
    fig = go.Figure()

    fig.update_layout(
        title=f'{ticker_symbol} Candlestick Chart',
        xaxis_title=f"Date:{date}",
        yaxis_title="Price",
        dragmode="pan",                 # ✅ allow panning
        xaxis=dict(
            type="category",
            rangeslider=dict(visible=False),
            tickmode="array",
            showticklabels=False,  # ✅ hides x-axis labels
        )
    )

    # Enable zoom in both directions
    fig.update_xaxes(fixedrange=False)
    fig.update_yaxes(fixedrange=False)

    return fig


def update_candlestick_data(
    fig: go.Figure,
    historical_data: pd.DataFrame
):
    candle = go.Candlestick(
        x=historical_data.index,
        open=historical_data["Open"],
        high=historical_data["High"],
        low=historical_data["Low"],
        close=historical_data["Close"],
        increasing_line_color="green",
        decreasing_line_color="red",
    )
    fig.add_trace(candle)
    return fig

def add_hline(fig: go.Figure,value,annot,color="blue",annot_pos="top left"):
    fig.add_hline(
      y= value,
      line=dict(color=color, width=1, dash="dash"),
      annotation_text=f"{annot}",
      annotation_position=annot_pos
    )
    return fig

def add_vline(fig: go.Figure,date):
    start_dt = pd.Timestamp(f"{date} 10:10:00").tz_localize("Asia/Kolkata")
    end_dt   = pd.Timestamp(f"{date} 10:15:00").tz_localize("Asia/Kolkata")

    fig.add_vrect(
      x0=start_dt,
      x1=end_dt,
      fillcolor="rgba(255,0,0,0.12)",  # translucent red band
      line_width=0,
      layer="below"
    )
    return fig


def add_marker(fig: go.Figure,time,price,text):
    fig.add_annotation(
      x=time,
      y=price,
      text=text,
      showarrow=True,
      arrowhead=2,
      arrowsize=1.2,
      arrowcolor="crimson",
      ax=-40, ay=-40,
      bgcolor="rgba(255,255,255,0.8)",
      bordercolor="crimson",
      borderwidth=1,
      font=dict(size=12, color="black"),
      xref="x", yref="y"
    )
    return fig



In [16]:
#Strategy1_testing

def strategy1(ticker_symbol, date):
  fig = create_candlestick_figure(ticker_symbol,date)

  # --------- LEVELS (Higher TF) ---------
  ht_data = get_data(ticker_symbol, date, "1h")
  support    = ht_data["High"].iloc[0]
  resistance = ht_data["Low"].iloc[0]

  # --------- LOWER TF DATA ---------
  data = get_data(ticker_symbol, date, "5m");
  fig = update_candlestick_data(fig, historical_data=data)

  fig = add_hline(fig,support,"support")
  fig = add_hline(fig,resistance,"resistance")
  fig = add_vline(fig,date)

  start_tf = pd.Timestamp(f"{date} 10:15:00").tz_localize("Asia/Kolkata")

  # --------- STATE ---------
  short_state = {"breakout": False, "risk": None}
  long_state  = {"breakout": False, "risk": None}

  # --------- ITERATION ---------
  for row in data.itertuples():
    index = row.Index
    if index < start_tf:
        continue

    close, high, low = row.Close, row.High, row.Low

    # ===== SHORT SETUP =====
    if not short_state["breakout"]:
      if close > support:
        short_state["breakout"] = True
        short_state["risk"] = high
        fig = add_marker(fig, index, close, "Breakout (Short)")
    else:
        short_state["risk"] = max(short_state["risk"], high)
        if close < support:
          stop = short_state["risk"]
          target = close + (close - stop) * 2

          fig = add_marker(fig, index, close, "Entry (Short)")
          fig = add_hline(fig, stop, "Stoploss", "red", "top right")
          fig = add_hline(fig, target, "Target", "green", "top right")
          break

    # ===== LONG SETUP =====
    if not long_state["breakout"]:
      if close < resistance:
        long_state["breakout"] = True
        long_state["risk"] = low
        fig = add_marker(fig, index, close, "Breakout (Long)")
    else:
      long_state["risk"] = min(long_state["risk"], low)
      if close > resistance:
        stop = long_state["risk"]
        target = close - (stop - close) * 2

        fig = add_marker(fig, index, close, "Entry (Long)")
        fig = add_hline(fig, stop, "Stoploss", "red", "top right")
        fig = add_hline(fig, target, "Target", "green", "top right")
        break

  return fig


In [17]:
tz="Asia/Kolkata"

def run_strategy_range(ticker_symbol, days_back, pause_each=True):
    today = pd.Timestamp.now(tz=tz).normalize()
    start = today - pd.Timedelta(days=days_back)
    dates = pd.date_range(start=start, end=today, freq="D", tz=tz)

    total = len(dates)
    print(f"Running from {dates[0].date()} to {dates[-1].date()} ({total} days)…")

    auto = False
    for i, dt in enumerate(dates, start=1):
        date_str = dt.strftime("%Y-%m-%d")  # matches your strategy1 signature
        print(f"\n[{i}/{total}] strategy1({ticker_symbol!r}, {date_str})")

        try:
            fig = strategy1(ticker_symbol, date_str)  # <-- your existing function
            fig.show(config={"scrollZoom": True})
        except Exception as e:
            pass



In [18]:
#Main entry
ticker_symbol = "BRITANNIA.NS"
date = "2025-12-19"

run_strategy_range(ticker_symbol,10)
#fig = strategy1(ticker_symbol, date);
#fig.show(config={"scrollZoom": True})


Running from 2025-12-15 to 2025-12-25 (11 days)…

[1/11] strategy1('BRITANNIA.NS', 2025-12-15)



[2/11] strategy1('BRITANNIA.NS', 2025-12-16)



[3/11] strategy1('BRITANNIA.NS', 2025-12-17)



[4/11] strategy1('BRITANNIA.NS', 2025-12-18)



[5/11] strategy1('BRITANNIA.NS', 2025-12-19)


...


[6/11] strategy1('BRITANNIA.NS', 2025-12-20)

[7/11] strategy1('BRITANNIA.NS', 2025-12-21)

[8/11] strategy1('BRITANNIA.NS', 2025-12-22)



[9/11] strategy1('BRITANNIA.NS', 2025-12-23)



[10/11] strategy1('BRITANNIA.NS', 2025-12-24)



[11/11] strategy1('BRITANNIA.NS', 2025-12-25)


ERROR:yfinance:$BRITANNIA.NS: possibly delisted; no price data found  (1h 2025-12-25 00:00:00 -> 2025-12-26 00:00:00)
